# Centro Democracia y Comunidad

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_articles_from_url(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)
        
        # Realizar scroll hasta el final de la página (si es necesario)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Esperamos un poco para que se cargue el contenido

        # Obtener todos los artículos dentro de los divs con la clase 'pt-cv-content-item'
        articles = driver.find_elements(By.CSS_SELECTOR, "div.pt-cv-content-item")

        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, ".pt-cv-thumb-wrapper img")
                image_url = img.get_attribute("src")
            except:
                image_url = None

            # Título y enlace
            try:
                title_element = article.find_element(By.CSS_SELECTOR, ".pt-cv-title a")
                title = title_element.text.strip()
                link = title_element.get_attribute("href")
            except:
                title = "No disponible"
                link = "No disponible"

            # Categorías (pueden haber varias, las extraemos todas)
            categories = []
            try:
                category_elements = article.find_elements(By.CSS_SELECTOR, ".pt-cv-taxoterm a")
                for category in category_elements:
                    categories.append(category.text.strip())
            except:
                categories = ["No especificada"]

            # Contenido (extracto del texto)
            try:
                content = article.find_element(By.CSS_SELECTOR, ".pt-cv-content").text.strip()
            except:
                content = None

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Categorías': ", ".join(categories),  # Unimos las categorías con coma
                'URL Imagen': image_url,
                'Contenido': content,
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()
    
def extract_articles_from_multiple_pages(driver, base_url, max_pages=10, existing_df=None):
    """
    Extrae información de artículos desde varias páginas de un sitio web y concatena los resultados en un DataFrame.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        base_url (str): URL base de la página de artículos (sin el parámetro de página).
        max_pages (int, optional): Número máximo de páginas a recorrer (por defecto 10). 
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas.
    """
    all_data = existing_df if existing_df is not None else pd.DataFrame()

    for page_num in range(1, max_pages + 1):
        try:
            # Construir la URL con el número de página
            url = f"{base_url}?_page={page_num}"
            print(f"Extrayendo artículos de {url}...")

            # Extraer los artículos de la página actual
            page_df = extract_articles_from_url(driver, url, existing_df=None)

            # Concatenar los artículos extraídos a los datos existentes
            all_data = pd.concat([all_data, page_df], ignore_index=True)
            
            # Esperar un poco entre las páginas para no sobrecargar el servidor
            time.sleep(3)

            # Verificar si la página está vacía o no contiene artículos (en ese caso, detener la iteración)
            if page_df.empty:
                print("No se encontraron más artículos en esta página. Terminando la extracción.")
                break

        except Exception as e:
            print(f"Error en la página {page_num}: {e}")
            continue  # Si ocurre un error en una página, seguir con la siguiente

    return all_data

In [3]:
# Crear un driver
driver = create_driver(headless=False)

# Definir la URL base sin el parámetro de página
base_url = "https://www.cdc.cl/centro-de-documentacion/"

# Extraer los artículos de las primeras 5 páginas
cdc = extract_articles_from_multiple_pages(driver, base_url, max_pages=7)

# Ver los primeros registros extraídos
cdc

# Guardar el DataFrame en un archivo CSV si lo deseas
#all_articles_df.to_csv('articulos_cdc.csv', index=False)

Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=1...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=2...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=3...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=4...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=5...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=6...
Extrayendo artículos de https://www.cdc.cl/centro-de-documentacion/?_page=7...


,Título,Enlace,Categorías,URL Imagen,Contenido,Fecha Consulta
0,“ES NECESARIO UN ENFOQUE INTEGRAL DE LA GESTIÓ...,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,"Humanismo Cristiano, Otros, Reflexión y debate...",https://www.cdc.cl/wp-content/uploads/2024/07/...,GESTIÓN Y ABORDAJE DE LA MIGRACIÓN A NIVEL MUN...,2025-01-09 14:21:58
1,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,"Reflexión y debate, seguridad y derechos humanos",https://www.cdc.cl/wp-content/uploads/2024/04/...,"Desde el 2004, nuestro país cuenta con un Sist...",2025-01-09 14:21:59
2,En memoria de Jacques Maritain,https://www.cdc.cl/publicaciones/historia/en-m...,Humanismo Cristiano,https://www.cdc.cl/wp-content/uploads/2023/08/...,"Maritain fue un insigne filósofo, pensador, es...",2025-01-09 14:21:59
3,Discurso del presidente Patricio Aylwin cuando...,https://www.cdc.cl/publicaciones/historia/disc...,Humanismo Cristiano,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Hace más de una década, cuando se inauguró el ...",2025-01-09 14:21:59
4,Boletín Legislativo,https://www.cdc.cl/publicaciones/bl/boletin-le...,Legislativo y Constitucional,https://www.cdc.cl/wp-content/plugins/pt-conte...,Revisa nuestro Boletín Legislativo de esta sem...,2025-01-09 14:21:59
...,...,...,...,...,...,...
112,Un Chile abierto: propuesta para una nueva ley...,https://www.cdc.cl/publicaciones/un-chile-abie...,Perfeccionamiento de la Democracia,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Autor: Centro Democracia y Comunidad, de la Or...",2025-01-09 14:23:22
113,El desafío de la ciudad y calidad de vida,https://www.cdc.cl/publicaciones/el-desafio-de...,Economía con equidad,https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Iván Poduje y Marcelo Ruiz Año: 2013 Br...,2025-01-09 14:23:22
114,La Solidaridad social como carencia y oportuni...,https://www.cdc.cl/publicaciones/la-solidarida...,Economía con equidad,https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Felipe Zaldivia Fuenzalida Año: 2014 Br...,2025-01-09 14:23:22
115,Hombre-Medio Ambiente: Fundamentos para una re...,https://www.cdc.cl/publicaciones/hombre-medio-...,"Humanismo Cristiano, Medio Ambiente",https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Yasmina Viera Bernal Año: 2014 Breve de...,2025-01-09 14:23:22


# Prueba 2

In [64]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

def extract_ficha_info(driver, url):
    """
    Extrae la información de la ficha de un documento específico
    """
    try:
        print(f"Procesando: {url}")
        driver.get(url)
        
        # Dar tiempo para que cargue el contenido
        time.sleep(1)
        
        # Inicializar diccionario para almacenar la información
        info = {}
        
        # Buscar el contenido completo de la ficha
        try:
            contenido_ficha = driver.find_element(By.CLASS_NAME, "kt-inside-inner-col")
            texto_ficha = contenido_ficha.text
        except NoSuchElementException:
            texto_ficha = ""
        
        # Mapeo de campos a buscar
        campos = {
            "Título": "",
            "Autor(es)": "taxonomy-autor",
            "Colección": "taxonomy-coleccion",
            "Tema(s)": "taxonomy-tema",
            "Fecha": "",
            "Palabras clave": "taxonomy-post_tag"
        }
        
        # Extraer información para cada campo
        for campo, clase in campos.items():
            try:
                if clase:  # Si tiene una clase específica
                    elemento = driver.find_element(By.CLASS_NAME, clase)
                    info[campo] = elemento.text.strip()
                else:  # Para campos sin clase específica
                    # Buscar el campo por su etiqueta dentro del texto de la ficha
                    # Aquí estamos buscando si el campo aparece dentro del texto y luego extraemos el valor correspondiente
                    if campo in texto_ficha:
                        # Extraer el valor que sigue a la etiqueta de campo
                        start_index = texto_ficha.find(campo + ":") + len(campo) + 1
                        if start_index != -1:
                            end_index = texto_ficha.find("\n", start_index)
                            if end_index == -1:  # Si no hay salto de línea, tomamos hasta el final
                                valor = texto_ficha[start_index:].strip()
                            else:
                                valor = texto_ficha[start_index:end_index].strip()
                            info[campo] = valor
                        else:
                            info[campo] = "No encontrado"
                    else:
                        info[campo] = "No encontrado"
            except NoSuchElementException:
                info[campo] = "No encontrado"
                
    except Exception as e:
        print(f"Error procesando {url}: {str(e)}")
        return {campo: "Error" for campo in campos.keys()}
    
    return info

def process_dataframe(df, headless=True):
    """
    Procesa el dataframe completo, extrayendo información para cada enlace
    """
    # Crear una copia del dataframe original
    df_nuevo = df.copy()
    
    # Inicializar el navegador con opción headless
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('--headless')  # Modo sin interfaz gráfica
    driver = webdriver.Chrome(options=options)
    
    try:
        # Procesar cada enlace
        resultados = []
        for url in df['Enlace']:
            info = extract_ficha_info(driver, url)
            resultados.append(info)
        
        # Convertir resultados a dataframe y unir con el original
        df_info = pd.DataFrame(resultados)
        df_nuevo = pd.concat([df_nuevo, df_info], axis=1)
        
    finally:
        driver.quit()
    
    return df_nuevo

# Ejemplo de uso:
df_resultado = process_dataframe(cdc[:6], headless=False)
df_resultado

Procesando: https://www.cdc.cl/cdc/es-necesario-un-enfoque-integral-de-la-gestion-de-movilidad-humana-a-nivel-de-estado-con-los-tres-niveles-de-gobierno-nacional-regional-y-local/
Procesando: https://www.cdc.cl/otras/crisis-de-seguridad-la-inteligencia-como-herramienta-indispensable/
Procesando: https://www.cdc.cl/publicaciones/historia/en-memoria-de-jacques-maritain/
Procesando: https://www.cdc.cl/publicaciones/historia/discurso-del-presidente-patricio-aylwin-cuando-nacio-cdc/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-23/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-22/


,Título,Enlace,Categorías,URL Imagen,Contenido,Fecha Consulta,Título,Autor(es),Colección,Tema(s),Fecha,Palabras clave
0,“ES NECESARIO UN ENFOQUE INTEGRAL DE LA GESTIÓ...,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,"Humanismo Cristiano, Otros, Reflexión y debate...",https://www.cdc.cl/wp-content/uploads/2024/07/...,GESTIÓN Y ABORDAJE DE LA MIGRACIÓN A NIVEL MUN...,2025-01-03 23:00:33,No encontrado,No encontrado,No encontrado,No encontrado,No encontrado,No encontrado
1,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,"Reflexión y debate, seguridad y derechos humanos",https://www.cdc.cl/wp-content/uploads/2024/04/...,"Desde el 2004, nuestro país cuenta con un Sist...",2025-01-03 23:00:33,No encontrado,No encontrado,No encontrado,No encontrado,No encontrado,No encontrado
2,En memoria de Jacques Maritain,https://www.cdc.cl/publicaciones/historia/en-m...,Humanismo Cristiano,https://www.cdc.cl/wp-content/uploads/2023/08/...,"Maritain fue un insigne filósofo, pensador, es...",2025-01-03 23:00:33,,No encontrado,No encontrado,Humanismo Cristiano,,"Análisis político, Ideas en comunidad, Portada"
3,Discurso del presidente Patricio Aylwin cuando...,https://www.cdc.cl/publicaciones/historia/disc...,Humanismo Cristiano,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Hace más de una década, cuando se inauguró el ...",2025-01-03 23:00:33,,No encontrado,No encontrado,Humanismo Cristiano,,Portada
4,Boletín Legislativo,https://www.cdc.cl/publicaciones/bl/boletin-le...,Legislativo y Constitucional,https://www.cdc.cl/wp-content/plugins/pt-conte...,Revisa nuestro Boletín Legislativo de esta sem...,2025-01-03 23:00:33,,Centro Democracia y Comunidad,La Semana en la Cámara,Legislativo y Constitucional,,"Boletín Legislativo, derechos de aseo, ENAP, T..."
5,Boletín Legislativo,https://www.cdc.cl/publicaciones/bl/boletin-le...,Legislativo y Constitucional,https://www.cdc.cl/wp-content/plugins/pt-conte...,Revisa nuestro Boletín Legislativo de esta sem...,2025-01-03 23:00:33,,No encontrado,No encontrado,Legislativo y Constitucional,,No encontrado


In [68]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

def extract_ficha_info(driver, url):
    """
    Extrae la información de la ficha de un documento específico
    """
    try:
        print(f"Procesando: {url}")
        driver.get(url)
        
        # Dar tiempo para que cargue el contenido
        time.sleep(1)
        
        # Inicializar diccionario para almacenar la información
        info = {}
        
        # Mapeo de campos a buscar
        campos = {
            "Título": "",
            "Autor(es)": "taxonomy-autor",
            "Colección": "taxonomy-coleccion",
            "Tema(s)": "taxonomy-tema",
            "Fecha": "",
            "Palabras clave": "taxonomy-post_tag"
        }
        
        # Extraer información para cada campo
        for campo, clase in campos.items():
            try:
                if clase:  # Si tiene una clase específica (como en "Autor(es)", "Colección", etc.)
                    elemento = driver.find_element(By.CLASS_NAME, clase)
                    info[campo] = elemento.text.strip()
                else:  # Para campos como "Título" y "Fecha" que no tienen clase específica
                    # Buscar el campo por su texto dentro de los elementos HTML
                    rows = driver.find_elements(By.CLASS_NAME, "kt-row-column-wrap")
                    for row in rows:
                        if campo in row.text:  # Si encontramos el campo en el texto
                            # Buscar el valor que sigue al campo, podría ser en un <p> o en otro contenedor
                            try:
                                # Buscamos el siguiente <p> después del campo clave
                                valor_elemento = row.find_element(By.XPATH, f".//strong[text()='{campo}:']/following::p[1]")
                                info[campo] = valor_elemento.text.strip()
                            except NoSuchElementException:
                                # Si no se encuentra el valor, asignamos un valor por defecto
                                info[campo] = "No encontrado"
            except NoSuchElementException:
                info[campo] = "No encontrado"
                
    except Exception as e:
        print(f"Error procesando {url}: {str(e)}")
        return {campo: "Error" for campo in campos.keys()}
    
    return info

def process_dataframe(df, headless=True):
    """
    Procesa el dataframe completo, extrayendo información para cada enlace
    """
    # Crear una copia del dataframe original
    df_nuevo = df.copy()
    
    # Inicializar el navegador con opción headless
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('--headless')  # Modo sin interfaz gráfica
    driver = webdriver.Chrome(options=options)
    
    try:
        # Procesar cada enlace
        resultados = []
        for url in df['Enlace']:
            info = extract_ficha_info(driver, url)
            resultados.append(info)
        
        # Convertir resultados a dataframe y unir con el original
        df_info = pd.DataFrame(resultados)
        df_nuevo = pd.concat([df_nuevo, df_info], axis=1)
        
    finally:
        driver.quit()
    
    return df_nuevo

# Ejemplo de uso:
df_resultado = process_dataframe(cdc, headless=False)
df_resultado

Procesando: https://www.cdc.cl/cdc/es-necesario-un-enfoque-integral-de-la-gestion-de-movilidad-humana-a-nivel-de-estado-con-los-tres-niveles-de-gobierno-nacional-regional-y-local/
Procesando: https://www.cdc.cl/otras/crisis-de-seguridad-la-inteligencia-como-herramienta-indispensable/
Procesando: https://www.cdc.cl/publicaciones/historia/en-memoria-de-jacques-maritain/
Procesando: https://www.cdc.cl/publicaciones/historia/discurso-del-presidente-patricio-aylwin-cuando-nacio-cdc/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-23/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-22/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-21/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-20/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-19/
Procesando: https://www.cdc.cl/publicaciones/boletin-legislativo-18/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-17/
Procesando: http

,Título,Enlace,Categorías,URL Imagen,Contenido,Fecha Consulta,Autor(es),Colección,Tema(s),Palabras clave,Título,Fecha
0,“ES NECESARIO UN ENFOQUE INTEGRAL DE LA GESTIÓ...,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,"Humanismo Cristiano, Otros, Reflexión y debate...",https://www.cdc.cl/wp-content/uploads/2024/07/...,GESTIÓN Y ABORDAJE DE LA MIGRACIÓN A NIVEL MUN...,2025-01-03 23:00:33,No encontrado,No encontrado,No encontrado,No encontrado,NaN,NaN
1,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,"Reflexión y debate, seguridad y derechos humanos",https://www.cdc.cl/wp-content/uploads/2024/04/...,"Desde el 2004, nuestro país cuenta con un Sist...",2025-01-03 23:00:33,No encontrado,No encontrado,No encontrado,No encontrado,NaN,NaN
2,En memoria de Jacques Maritain,https://www.cdc.cl/publicaciones/historia/en-m...,Humanismo Cristiano,https://www.cdc.cl/wp-content/uploads/2023/08/...,"Maritain fue un insigne filósofo, pensador, es...",2025-01-03 23:00:33,No encontrado,No encontrado,Humanismo Cristiano,"Análisis político, Ideas en comunidad, Portada",,
3,Discurso del presidente Patricio Aylwin cuando...,https://www.cdc.cl/publicaciones/historia/disc...,Humanismo Cristiano,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Hace más de una década, cuando se inauguró el ...",2025-01-03 23:00:33,No encontrado,No encontrado,Humanismo Cristiano,Portada,,
4,Boletín Legislativo,https://www.cdc.cl/publicaciones/bl/boletin-le...,Legislativo y Constitucional,https://www.cdc.cl/wp-content/plugins/pt-conte...,Revisa nuestro Boletín Legislativo de esta sem...,2025-01-03 23:00:33,Centro Democracia y Comunidad,La Semana en la Cámara,Legislativo y Constitucional,"Boletín Legislativo, derechos de aseo, ENAP, T...",Boletín legislativo N° 24,20/01/2023
...,...,...,...,...,...,...,...,...,...,...,...,...
112,Un Chile abierto: propuesta para una nueva ley...,https://www.cdc.cl/publicaciones/un-chile-abie...,Perfeccionamiento de la Democracia,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Autor: Centro Democracia y Comunidad, de la Or...",2025-01-03 23:01:40,No encontrado,No encontrado,Perfeccionamiento de la Democracia,No encontrado,,
113,El desafío de la ciudad y calidad de vida,https://www.cdc.cl/publicaciones/el-desafio-de...,Economía con equidad,https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Iván Poduje y Marcelo Ruiz Año: 2013 Br...,2025-01-03 23:01:40,No encontrado,No encontrado,Economía con equidad,No encontrado,,
114,La Solidaridad social como carencia y oportuni...,https://www.cdc.cl/publicaciones/la-solidarida...,Economía con equidad,https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Felipe Zaldivia Fuenzalida Año: 2014 Br...,2025-01-03 23:01:40,No encontrado,No encontrado,Economía con equidad,No encontrado,,
115,Hombre-Medio Ambiente: Fundamentos para una re...,https://www.cdc.cl/publicaciones/hombre-medio-...,"Humanismo Cristiano, Medio Ambiente",https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Yasmina Viera Bernal Año: 2014 Breve de...,2025-01-03 23:01:40,No encontrado,No encontrado,"Humanismo Cristiano, Medio Ambiente",No encontrado,,


In [70]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

def extract_ficha_info(driver, url):
    """
    Extrae la información de la ficha de un documento específico
    """
    try:
        print(f"Procesando: {url}")
        driver.get(url)
        
        # Dar tiempo para que cargue el contenido
        time.sleep(1)
        
        # Inicializar diccionario para almacenar la información
        info = {}
        
        # Mapeo de campos a buscar
        campos = {
            "Título": "",
            "Autor(es)": "taxonomy-autor",
            "Colección": "taxonomy-coleccion",
            "Tema(s)": "taxonomy-tema",
            "Fecha": "",
            "Palabras clave": "taxonomy-post_tag"
        }
        
        # Extraer información para cada campo
        for campo, clase in campos.items():
            try:
                if clase:  # Si tiene una clase específica (como en "Autor(es)", "Colección", etc.)
                    elemento = driver.find_element(By.CLASS_NAME, clase)
                    info[campo] = elemento.text.strip() if elemento.text.strip() else "No disponible"
                else:  # Para campos como "Título" y "Fecha" que no tienen clase específica
                    rows = driver.find_elements(By.CLASS_NAME, "kt-row-column-wrap")
                    encontrado = False
                    for row in rows:
                        if campo in row.text:  # Si encontramos el campo en el texto
                            try:
                                # Intentar obtener el valor usando un XPath
                                valor_elemento = row.find_element(By.XPATH, f".//strong[text()='{campo}:']/following::p[1]")
                                valor_texto = valor_elemento.text.strip()
                                
                                # Verificar si el valor no está vacío
                                if valor_texto:
                                    info[campo] = valor_texto
                                else:
                                    info[campo] = "No disponible"
                                encontrado = True
                                break
                            except NoSuchElementException:
                                # Si no se encuentra el valor, lo marcamos como no encontrado
                                info[campo] = "No disponible"
                                encontrado = True
                                break
                    
                    # Si no se encuentra el valor dentro de las filas esperadas, asignamos "No disponible"
                    if not encontrado:
                        info[campo] = "No disponible"
                        
            except NoSuchElementException:
                info[campo] = "No disponible"
                
    except Exception as e:
        print(f"Error procesando {url}: {str(e)}")
        return {campo: "Error" for campo in campos.keys()}
    
    return info

def process_dataframe(df, headless=True):
    """
    Procesa el dataframe completo, extrayendo información para cada enlace
    """
    # Crear una copia del dataframe original
    df_nuevo = df.copy()
    
    # Inicializar el navegador con opción headless
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('--headless')  # Modo sin interfaz gráfica
    driver = webdriver.Chrome(options=options)
    
    try:
        # Procesar cada enlace
        resultados = []
        for url in df['Enlace']:
            info = extract_ficha_info(driver, url)
            resultados.append(info)
        
        # Convertir resultados a dataframe y unir con el original
        df_info = pd.DataFrame(resultados)
        df_nuevo = pd.concat([df_nuevo, df_info], axis=1)
        
    finally:
        driver.quit()
    
    return df_nuevo

# Ejemplo de uso:
df_resultado = process_dataframe(cdc, headless=False)
df_resultado

Procesando: https://www.cdc.cl/cdc/es-necesario-un-enfoque-integral-de-la-gestion-de-movilidad-humana-a-nivel-de-estado-con-los-tres-niveles-de-gobierno-nacional-regional-y-local/
Procesando: https://www.cdc.cl/otras/crisis-de-seguridad-la-inteligencia-como-herramienta-indispensable/
Procesando: https://www.cdc.cl/publicaciones/historia/en-memoria-de-jacques-maritain/
Procesando: https://www.cdc.cl/publicaciones/historia/discurso-del-presidente-patricio-aylwin-cuando-nacio-cdc/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-23/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-22/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-21/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-20/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-19/
Procesando: https://www.cdc.cl/publicaciones/boletin-legislativo-18/
Procesando: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-17/
Procesando: http

,Título,Enlace,Categorías,URL Imagen,Contenido,Fecha Consulta,Título,Autor(es),Colección,Tema(s),Fecha,Palabras clave
0,“ES NECESARIO UN ENFOQUE INTEGRAL DE LA GESTIÓ...,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,"Humanismo Cristiano, Otros, Reflexión y debate...",https://www.cdc.cl/wp-content/uploads/2024/07/...,GESTIÓN Y ABORDAJE DE LA MIGRACIÓN A NIVEL MUN...,2025-01-03 23:00:33,No disponible,No disponible,No disponible,No disponible,No disponible,No disponible
1,Crisis de seguridad: La inteligencia como herr...,https://www.cdc.cl/otras/crisis-de-seguridad-l...,"Reflexión y debate, seguridad y derechos humanos",https://www.cdc.cl/wp-content/uploads/2024/04/...,"Desde el 2004, nuestro país cuenta con un Sist...",2025-01-03 23:00:33,No disponible,No disponible,No disponible,No disponible,No disponible,No disponible
2,En memoria de Jacques Maritain,https://www.cdc.cl/publicaciones/historia/en-m...,Humanismo Cristiano,https://www.cdc.cl/wp-content/uploads/2023/08/...,"Maritain fue un insigne filósofo, pensador, es...",2025-01-03 23:00:33,No disponible,No disponible,No disponible,Humanismo Cristiano,No disponible,"Análisis político, Ideas en comunidad, Portada"
3,Discurso del presidente Patricio Aylwin cuando...,https://www.cdc.cl/publicaciones/historia/disc...,Humanismo Cristiano,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Hace más de una década, cuando se inauguró el ...",2025-01-03 23:00:33,No disponible,No disponible,No disponible,Humanismo Cristiano,No disponible,Portada
4,Boletín Legislativo,https://www.cdc.cl/publicaciones/bl/boletin-le...,Legislativo y Constitucional,https://www.cdc.cl/wp-content/plugins/pt-conte...,Revisa nuestro Boletín Legislativo de esta sem...,2025-01-03 23:00:33,Boletín legislativo N° 24,Centro Democracia y Comunidad,La Semana en la Cámara,Legislativo y Constitucional,20/01/2023,"Boletín Legislativo, derechos de aseo, ENAP, T..."
...,...,...,...,...,...,...,...,...,...,...,...,...
112,Un Chile abierto: propuesta para una nueva ley...,https://www.cdc.cl/publicaciones/un-chile-abie...,Perfeccionamiento de la Democracia,https://www.cdc.cl/wp-content/plugins/pt-conte...,"Autor: Centro Democracia y Comunidad, de la Or...",2025-01-03 23:01:40,No disponible,No disponible,No disponible,Perfeccionamiento de la Democracia,No disponible,No disponible
113,El desafío de la ciudad y calidad de vida,https://www.cdc.cl/publicaciones/el-desafio-de...,Economía con equidad,https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Iván Poduje y Marcelo Ruiz Año: 2013 Br...,2025-01-03 23:01:40,No disponible,No disponible,No disponible,Economía con equidad,No disponible,No disponible
114,La Solidaridad social como carencia y oportuni...,https://www.cdc.cl/publicaciones/la-solidarida...,Economía con equidad,https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Felipe Zaldivia Fuenzalida Año: 2014 Br...,2025-01-03 23:01:40,No disponible,No disponible,No disponible,Economía con equidad,No disponible,No disponible
115,Hombre-Medio Ambiente: Fundamentos para una re...,https://www.cdc.cl/publicaciones/hombre-medio-...,"Humanismo Cristiano, Medio Ambiente",https://www.cdc.cl/wp-content/plugins/pt-conte...,Autor: Yasmina Viera Bernal Año: 2014 Breve de...,2025-01-03 23:01:40,No disponible,No disponible,No disponible,"Humanismo Cristiano, Medio Ambiente",No disponible,No disponible


In [71]:
df_resultado.columns

Index(['Título', 'Enlace', 'Categorías', 'URL Imagen', 'Contenido',
       'Fecha Consulta', 'Título', 'Autor(es)', 'Colección', 'Tema(s)',
       'Fecha', 'Palabras clave'],
      dtype='object')

# Anterior (funciona ahi no mas)

In [51]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def obtener_informacion_sitios(df):
    """
    Itera sobre los enlaces en la columna 'Enlace' del DataFrame y extrae información
    de cada página web, incluyendo Título, Autor, Colección, Contenido y Fecha de publicación.

    Args:
    - df: DataFrame con una columna 'Enlace' que contiene los URLs de las páginas a procesar.

    Returns:
    - DataFrame con la información extraída de cada página.
    """
    # Lista para almacenar la información extraída
    resultados = []

    # Iteramos sobre cada enlace del DataFrame
    for index, row in df.iterrows():
        enlace = row['Enlace']
        print(f"Obteniendo datos de: {enlace}")

        # Realizamos una solicitud HTTP a la URL
        try:
            respuesta = requests.get(enlace)
            respuesta.raise_for_status()  # Levanta un error si la respuesta es mala (status code 4xx/5xx)
            soup = BeautifulSoup(respuesta.text, 'html.parser')

            # Intentamos extraer los datos relevantes del HTML
            titulo = obtener_texto_por_clase(soup, 'wp-block-heading', 'Título')
            autor = obtener_texto_por_clase(soup, 'wp-block-kadence-column', 'Autor')
            coleccion = obtener_texto_por_clase(soup, 'wp-block-kadence-column', 'Colección')
            contenido = obtener_contenido(soup)
            fecha_publicacion = obtener_fecha_publicacion(soup)

            # Almacenamos los resultados en un diccionario
            resultados.append({
                'Enlace': enlace,
                'Título': titulo,
                'Autor': autor,
                'Colección': coleccion,
                'Contenido': contenido,
                'Fecha de publicación': fecha_publicacion
            })
        except requests.exceptions.RequestException as e:
            print(f"Error al obtener el sitio {enlace}: {e}")
            resultados.append({
                'Enlace': enlace,
                'Título': None,
                'Autor': None,
                'Colección': None,
                'Contenido': None,
                'Fecha de publicación': None
            })

    # Convertimos la lista de resultados en un DataFrame
    df_resultado = pd.DataFrame(resultados)

    # Verificamos si la información se ha extraído correctamente
    print(df_resultado.head())  # Muestra las primeras filas para verificar la extracción

    return df_resultado


def obtener_texto_por_clase(soup, clase, etiqueta):
    """
    Extrae el texto de un elemento HTML específico dado su clase.
    Devuelve el texto si se encuentra, de lo contrario retorna 'No encontrado'.
    
    Args:
    - soup: objeto BeautifulSoup con el contenido HTML de la página.
    - clase: clase CSS para identificar el elemento.
    - etiqueta: texto identificador para extraer el dato específico.

    Returns:
    - str: el texto extraído o 'No encontrado' .
    """
    elementos = soup.find_all(class_=clase)
    for elemento in elementos:
        # Buscamos si contiene el texto relevante
        if etiqueta in elemento.text:
            texto = elemento.get_text(strip=True)
            return texto.split(":")[-1].strip() if ':' in texto else texto
    return 'No encontrado'


def obtener_contenido(soup):
    """
    Extrae el contenido principal de la página web, que está dentro de la clase 'entry-content wp-block-post-content'.
    
    Args:
    - soup: objeto BeautifulSoup con el contenido HTML de la página.
    
    Returns:
    - str: el contenido de la página o 'No encontrado' .
    """
    # Buscar el div que contiene el contenido (hacemos que la clase sea más flexible)
    contenido_div = soup.find('div', class_='entry-content')
    
    if contenido_div:
        # Extraemos todos los párrafos (<p>) dentro del contenido
        parrafos = contenido_div.find_all('p')
        if parrafos:
            contenido_texto = "\n".join([p.get_text(strip=True) for p in parrafos])
            return contenido_texto
        else:
            return 'No se encontraron párrafos.'
    else:
        return 'No se encontró el div de contenido.'


def obtener_fecha_publicacion(soup):
    """
    Extrae la fecha de publicación de la página web.
    Primero intenta buscar la fecha en un div con clase 'cs-entry__post-meta'.
    
    Args:
    - soup: objeto BeautifulSoup con el contenido HTML de la página.
    
    Returns:
    - str: la fecha de publicación o 'No encontrado' .
    """
    # Intentamos buscar la fecha en el div con clase 'cs-entry__post-meta' y 'cs-meta-date'
    fecha_div = soup.find('div', class_='cs-entry__post-meta')
    if fecha_div:
        fecha = fecha_div.find('div', class_='cs-meta-date')
        if fecha:
            return fecha.get_text(strip=True)
    
    # Si no se encuentra la fecha en el div esperado, buscamos en otros lugares
    fecha = soup.find_all('time')
    if fecha:
        return fecha[0].get_text(strip=True)
    
    # Si no encontramos nada, buscamos en posibles etiquetas de fecha o metadatos
    fecha = soup.find_all('span', {'class': 'fecha'})
    if fecha:
        return fecha[0].get_text(strip=True)

    # Si no se encuentra nada, retornamos 'No encontrado'
    return 'No encontrado'


# Llamamos a la función para obtener la información
df_resultado = obtener_informacion_sitios(cdc)

# Mostramos el DataFrame con los resultados
df_resultado

Obteniendo datos de: https://www.cdc.cl/cdc/es-necesario-un-enfoque-integral-de-la-gestion-de-movilidad-humana-a-nivel-de-estado-con-los-tres-niveles-de-gobierno-nacional-regional-y-local/
Obteniendo datos de: https://www.cdc.cl/otras/crisis-de-seguridad-la-inteligencia-como-herramienta-indispensable/
Obteniendo datos de: https://www.cdc.cl/publicaciones/historia/en-memoria-de-jacques-maritain/
Obteniendo datos de: https://www.cdc.cl/publicaciones/historia/discurso-del-presidente-patricio-aylwin-cuando-nacio-cdc/
Obteniendo datos de: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-23/
Obteniendo datos de: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-22/
Obteniendo datos de: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-21/
Obteniendo datos de: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-20/
Obteniendo datos de: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-19/
Obteniendo datos de: https://www.cdc.cl/publicaciones/boletin-legislativo-18

,Enlace,Título,Autor,Colección,Contenido,Fecha de publicación
0,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,No encontrado,No encontrado,No encontrado,GESTIÓN Y ABORDAJE DE LA MIGRACIÓN A NIVEL MUN...,07/10/2024
1,https://www.cdc.cl/otras/crisis-de-seguridad-l...,No encontrado,No encontrado,No encontrado,"\nDesde el 2004, nuestro país cuenta con un Si...",07/10/2024
2,https://www.cdc.cl/publicaciones/historia/en-m...,No encontrado,"Raymon Aaron. Sus tesis, de las que su introdu...","Raymon Aaron. Sus tesis, de las que su introdu...",Por María Teresa Compte Grau\nEl día 28 de abr...,No encontrado
3,https://www.cdc.cl/publicaciones/historia/disc...,No encontrado,,,"Hace más de una década, cuando se inauguró el ...",No encontrado
4,https://www.cdc.cl/publicaciones/bl/boletin-le...,No encontrado,Boletín Legislativo N°24.,Boletín Legislativo N°24.,Revisa nuestro Boletín Legislativo de esta sem...,No encontrado
...,...,...,...,...,...,...
112,https://www.cdc.cl/publicaciones/un-chile-abie...,No encontrado,El documento aborda los procesos migratorios q...,El documento aborda los procesos migratorios q...,"Autor:Centro Democracia y Comunidad, de la Org...",No encontrado
113,https://www.cdc.cl/publicaciones/el-desafio-de...,No encontrado,El documento detecta 6 problemas transversales...,El documento detecta 6 problemas transversales...,Autor:Iván Poduje y Marcelo RuizAño:2013\nBrev...,No encontrado
114,https://www.cdc.cl/publicaciones/la-solidarida...,No encontrado,El actual sistema económico nos plantea una se...,El actual sistema económico nos plantea una se...,Autor:Felipe Zaldivia FuenzalidaAño:2014\nBrev...,No encontrado
115,https://www.cdc.cl/publicaciones/hombre-medio-...,No encontrado,El ser humano se relaciona con todo lo que le ...,El ser humano se relaciona con todo lo que le ...,Autor:Yasmina Viera BernalAño:2014Breve descri...,No encontrado


In [ ]:
# Finalmente, cerrar el driver
driver.quit()

In [11]:
# Crear el driver
driver = create_driver(headless=False)

# URL de la página de la que quieres extraer artículos
base_url = "https://www.cdc.cl/actualidad/#tab-ideasencomunidad"

# Llamar a la función para extraer los artículos
all_articles_df2 = extract_articles_from_url(driver, base_url)

# Mostrar el DataFrame resultante
all_articles_df2

,Título,Enlace,Fecha Publicación,Categorías,URL Imagen,Contenido,Fecha Consulta
0,,https://www.cdc.cl/cdc/es-necesario-un-enfoque...,,Formación,https://www.cdc.cl/wp-content/uploads/2024/07/...,,2025-01-03 22:37:38
1,,https://www.cdc.cl/otras/crisis-de-seguridad-l...,,Formación,https://www.cdc.cl/wp-content/uploads/2024/04/...,,2025-01-03 22:37:38
2,,https://www.cdc.cl/ideas-en-comunidad-2/la-nec...,,Formación,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",,2025-01-03 22:37:38
3,,https://www.cdc.cl/ideas-en-comunidad-2/ideolo...,,Formación,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",,2025-01-03 22:37:38
4,,https://www.cdc.cl/ideas-en-comunidad-2/la-cri...,,Formación,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",,2025-01-03 22:37:38
...,...,...,...,...,...,...,...
75,Chile necesita un “sorpasso” del centro político,https://www.cdc.cl/ideas-en-comunidad-2/chile-...,10/16/2023,Formación,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Los centristas se manifiestan así a través de ...,2025-01-03 22:37:42
76,Veto presidencial: una nueva oportunidad para ...,https://www.cdc.cl/ideas-en-comunidad-2/veto-p...,10/12/2023,Formación,https://www.cdc.cl/wp-content/uploads/2023/10/...,El veto presidencial podría ser una oportunida...,2025-01-03 22:37:42
77,La encrucijada de la reconciliación,https://www.cdc.cl/ideas-en-comunidad-2/la-enc...,10/10/2023,Formación,https://www.cdc.cl/wp-content/uploads/2023/10/...,Las fuerzas progresistas se encuentran entonce...,2025-01-03 22:37:42
78,Escasez de liderazgos,https://www.cdc.cl/ideas-en-comunidad-2/escase...,10/06/2023,Formación,https://www.cdc.cl/wp-content/uploads/2023/10/...,La ausencia de liderazgos sólidos desde el cen...,2025-01-03 22:37:42


In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def obtener_informacion_ficha_selenium(driver):
    """
    Extrae la información de la ficha del documento si está presente en la página web usando Selenium.

    Args:
    - driver: WebDriver de Selenium.
    
    Returns:
    - dict: Diccionario con la información de la ficha o 'No encontrado'.
    """
    try:
        # Obtener los datos dentro de la ficha
        ficha_info = {}
        
        # Título
        ficha_info['Título Ficha'] = obtener_texto_por_clase_selenium(driver, 'wp-block-heading', 'Ficha del documento')

        # Autor
        autor_ficha = obtener_texto_por_clase_selenium(driver, 'wp-block-kadence-column', 'Autor')
        ficha_info['Autor Ficha'] = autor_ficha if autor_ficha != 'No encontrado' else 'No disponible'
        
        # Colección
        coleccion_ficha = obtener_texto_por_clase_selenium(driver, 'wp-block-kadence-column', 'Colección')
        ficha_info['Colección Ficha'] = coleccion_ficha if coleccion_ficha != 'No encontrado' else 'No disponible'
        
        return ficha_info
    
    except Exception as e:
        print(f"Error al obtener la ficha: {e}")
        return {'Título Ficha': 'No encontrado', 'Autor Ficha': 'No encontrado', 'Colección Ficha': 'No encontrado'}

def obtener_informacion_sitios_selenium(driver, df):
    """
    Itera sobre los enlaces en la columna 'Enlace' del DataFrame y extrae información
    de cada página web, incluyendo Título, Autor, Colección, Contenido, Fecha de publicación
    y datos de la ficha del documento (si está presente).

    Args:
    - df: DataFrame con una columna 'Enlace' que contiene los URLs de las páginas a procesar.

    Returns:
    - DataFrame con la información extraída de cada página.
    """
    resultados = []

    for index, row in df.iterrows():
        enlace = row['Enlace']
        print(f"Obteniendo datos de: {enlace}")

        try:
            # Navegar a la página usando Selenium
            driver.get(enlace)
            time.sleep(1)  # Esperamos un poco para que la página cargue completamente

            # Extraemos los datos relevantes usando Selenium
            titulo = obtener_texto_por_clase_selenium(driver, 'wp-block-heading', 'Título')
            autor = obtener_texto_por_clase_selenium(driver, 'wp-block-kadence-column', 'Autor')
            coleccion = obtener_texto_por_clase_selenium(driver, 'wp-block-kadence-column', 'Colección')
            contenido = obtener_contenido_selenium(driver)
            fecha_publicacion = obtener_fecha_publicacion_selenium(driver)

            # Extraer título, categorías y fecha del div .cs-entry__header-info
            header_title, header_categories, header_date = obtener_header_info_selenium(driver)

            # Intentar obtener los datos de la ficha (si está presente)
            ficha_info = obtener_informacion_ficha_selenium(driver)

            # Almacenamos los resultados en un diccionario, incluyendo la ficha
            resultados.append({
                'Enlace': enlace,
                'Título': titulo,
                'Autor': autor,
                'Colección': coleccion,
                'Contenido': contenido,
                'Fecha de publicación': fecha_publicacion,
                'Título Header': header_title,
                'Categorías Header': header_categories,
                'Fecha Header': header_date,
                'Título Ficha': ficha_info['Título Ficha'],
                'Autor Ficha': ficha_info['Autor Ficha'],
                'Colección Ficha': ficha_info['Colección Ficha']
            })
        except Exception as e:
            print(f"Error al obtener el sitio {enlace}: {e}")
            resultados.append({
                'Enlace': enlace,
                'Título': None,
                'Autor': None,
                'Colección': None,
                'Contenido': None,
                'Fecha de publicación': None,
                'Título Header': None,
                'Categorías Header': None,
                'Fecha Header': None,
                'Título Ficha': None,
                'Autor Ficha': None,
                'Colección Ficha': None
            })

    # Convertimos la lista de resultados en un DataFrame
    df_resultado = pd.DataFrame(resultados)

    print(df_resultado.head())  # Muestra las primeras filas para verificar la extracción

    return df_resultado

def obtener_texto_por_clase_selenium(driver, clase, etiqueta):
    """
    Extrae el texto de un elemento HTML específico dado su clase usando Selenium.
    
    Args:
    - driver: WebDriver de Selenium.
    - clase: clase CSS para identificar el elemento.
    - etiqueta: texto identificador para extraer el dato específico.

    Returns:
    - str: el texto extraído o 'No encontrado'.
    """
    try:
        elementos = driver.find_elements(By.CLASS_NAME, clase)
        for elemento in elementos:
            if etiqueta in elemento.text:
                texto = elemento.text.strip()
                return texto.split(":")[-1].strip() if ':' in texto else texto
        return 'No encontrado'
    except:
        return 'No encontrado'


def obtener_contenido_selenium(driver):
    """
    Extrae el contenido principal de la página web usando Selenium.
    
    Args:
    - driver: WebDriver de Selenium.
    
    Returns:
    - str: el contenido de la página o 'No encontrado'.
    """
    try:
        contenido_div = driver.find_element(By.CLASS_NAME, 'entry-content')
        parrafos = contenido_div.find_elements(By.TAG_NAME, 'p')
        contenido_texto = "\n".join([p.text.strip() for p in parrafos])
        return contenido_texto
    except:
        return 'No se encontró el div de contenido'


def obtener_fecha_publicacion_selenium(driver):
    """
    Extrae la fecha de publicación de la página web usando Selenium.
    
    Args:
    - driver: WebDriver de Selenium.
    
    Returns:
    - str: la fecha de publicación o 'No encontrado'.
    """
    try:
        fecha_div = driver.find_element(By.CLASS_NAME, 'cs-entry__post-meta')
        fecha = fecha_div.find_element(By.CLASS_NAME, 'cs-meta-date').text.strip()
        return fecha
    except:
        return 'No encontrado'


def obtener_header_info_selenium(driver):
    """
    Extrae el título, las categorías y la fecha de publicación desde el div '.cs-entry__header-info' usando Selenium.
    
    Args:
    - driver: WebDriver de Selenium.
    
    Returns:
    - tuple: (header_title, header_categories, header_date)
    """
    try:
        header_info = driver.find_element(By.CLASS_NAME, 'cs-entry__header-info')

        # Extraer título
        header_title = header_info.find_element(By.TAG_NAME, 'h1').text.strip()

        # Extraer categorías
        category_elements = header_info.find_elements(By.CSS_SELECTOR, '.cs-entry__category a')
        header_categories = ", ".join([cat.text.strip() for cat in category_elements])

        # Extraer fecha
        header_date = header_info.find_element(By.CLASS_NAME, 'cs-meta-date').text.strip()

        return header_title, header_categories, header_date
    except:
        return 'No encontrado', 'No encontrado', 'No encontrado'


# Crear el driver de Selenium
driver = create_driver(headless=False)

# Llamamos a la función para obtener la información
df_resultado = obtener_informacion_sitios_selenium(driver, cdc[:6])

# Mostramos el DataFrame con los resultados
df_resultado

Obteniendo datos de: https://www.cdc.cl/publicaciones/historia/discurso-del-presidente-patricio-aylwin-cuando-nacio-cdc/
Error al obtener el sitio https://www.cdc.cl/publicaciones/historia/discurso-del-presidente-patricio-aylwin-cuando-nacio-cdc/: 'Título Ficha'
Obteniendo datos de: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-23/
Error al obtener el sitio https://www.cdc.cl/publicaciones/bl/boletin-legislativo-23/: 'Título Ficha'
Obteniendo datos de: https://www.cdc.cl/publicaciones/bl/boletin-legislativo-22/
Error al obtener el sitio https://www.cdc.cl/publicaciones/bl/boletin-legislativo-22/: 'Título Ficha'
                                              Enlace Título Autor Colección  \
0  https://www.cdc.cl/publicaciones/historia/disc...   None  None      None   
1  https://www.cdc.cl/publicaciones/bl/boletin-le...   None  None      None   
2  https://www.cdc.cl/publicaciones/bl/boletin-le...   None  None      None   

  Contenido Fecha de publicación Título Header Categor

,Enlace,Título,Autor,Colección,Contenido,Fecha de publicación,Título Header,Categorías Header,Fecha Header,Título Ficha,Autor Ficha,Colección Ficha
0,https://www.cdc.cl/publicaciones/historia/disc...,None,None,None,None,None,None,None,None,None,None,None
1,https://www.cdc.cl/publicaciones/bl/boletin-le...,None,None,None,None,None,None,None,None,None,None,None
2,https://www.cdc.cl/publicaciones/bl/boletin-le...,None,None,None,None,None,None,None,None,None,None,None


In [45]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def obtener_informacion_ficha_selenium(driver):
    """
    Extrae la información de la ficha del documento si está presente en la página web usando Selenium.

    Args:
    - driver: WebDriver de Selenium.
    
    Returns:
    - dict: Diccionario con la información de la ficha o 'No encontrado' si algún dato no está disponible.
    """
    try:
        # Obtener los datos dentro de la ficha
        ficha_info = {}
        
        # Título Ficha: Se debe ajustar según la estructura de la página
        ficha_info['Título Ficha'] = obtener_texto_por_clase_selenium(driver, 'wp-block-heading', 'Ficha del documento')

        # Autor: Cambiar 'wp-block-kadence-column' a la clase correcta, si es necesario
        autor_ficha = obtener_texto_por_clase_selenium(driver, 'wp-block-kadence-column', 'Autor')
        ficha_info['Autor Ficha'] = autor_ficha if autor_ficha != 'No encontrado' else 'No disponible'
        
        # Colección: Cambiar la clase a la adecuada, si es necesario
        coleccion_ficha = obtener_texto_por_clase_selenium(driver, 'wp-block-kadence-column', 'Colección')
        ficha_info['Colección Ficha'] = coleccion_ficha if coleccion_ficha != 'No encontrado' else 'No disponible'
        
        return ficha_info
    
    except Exception as e:
        print(f"Error al obtener la ficha: {e}")
        return {'Título Ficha': 'No encontrado', 'Autor Ficha': 'No encontrado', 'Colección Ficha': 'No encontrado'}

def obtener_informacion_sitios_selenium(driver, df):
    """
    Itera sobre los enlaces en la columna 'Enlace' del DataFrame y extrae información
    de cada página web, incluyendo Título, Autor, Colección, Contenido, Fecha de publicación
    y datos de la ficha del documento (si está presente).

    Args:
    - df: DataFrame con una columna 'Enlace' que contiene los URLs de las páginas a procesar.

    Returns:
    - DataFrame con la información extraída de cada página.
    """
    resultados = []

    for index, row in df.iterrows():
        enlace = row['Enlace']
        print(f"Obteniendo datos de: {enlace}")

        try:
            # Navegar a la página usando Selenium
            driver.get(enlace)
            time.sleep(3)  # Esperamos un poco para que la página cargue completamente

            # Extraemos los datos relevantes usando Selenium
            titulo = obtener_texto_por_clase_selenium(driver, 'wp-block-heading', 'Título')
            autor = obtener_texto_por_clase_selenium(driver, 'wp-block-kadence-column', 'Autor')
            coleccion = obtener_texto_por_clase_selenium(driver, 'wp-block-kadence-column', 'Colección')
            contenido = obtener_contenido_selenium(driver)
            fecha_publicacion = obtener_fecha_publicacion_selenium(driver)

            # Extraer título, categorías y fecha del div .cs-entry__header-info
            header_title, header_categories, header_date = obtener_header_info_selenium(driver)

            # Intentar obtener los datos de la ficha (si está presente)
            ficha_info = obtener_informacion_ficha_selenium(driver)

            # Almacenamos los resultados en un diccionario, incluyendo la ficha
            resultados.append({
                'Enlace': enlace,
                'Título': titulo,
                'Autor': autor,
                'Colección': coleccion,
                'Contenido': contenido,
                'Fecha de publicación': fecha_publicacion,
                'Título Header': header_title,
                'Categorías Header': header_categories,
                'Fecha Header': header_date,
                'Título Ficha': ficha_info['Título Ficha'],
                'Autor Ficha': ficha_info['Autor Ficha'],
                'Colección Ficha': ficha_info['Colección Ficha']
            })
        except Exception as e:
            print(f"Error al obtener el sitio {enlace}: {e}")
            resultados.append({
                'Enlace': enlace,
                'Título': None,
                'Autor': None,
                'Colección': None,
                'Contenido': None,
                'Fecha de publicación': None,
                'Título Header': None,
                'Categorías Header': None,
                'Fecha Header': None,
                'Título Ficha': None,
                'Autor Ficha': None,
                'Colección Ficha': None
            })

    # Convertimos la lista de resultados en un DataFrame
    df_resultado = pd.DataFrame(resultados)

    print(df_resultado.head())  # Muestra las primeras filas para verificar la extracción

    return df_resultado

def obtener_texto_por_clase_selenium(driver, clase, etiqueta):
    """
    Extrae el texto de un elemento HTML específico dado su clase usando Selenium.
    
    Args:
    - driver: WebDriver de Selenium.
    - clase: clase CSS para identificar el elemento.
    - etiqueta: texto identificador para extraer el dato específico.

    Returns:
    - str: el texto extraído o 'No encontrado' si no se encuentra el elemento.
    """
    try:
        # Usamos find_elements para obtener todos los elementos con la clase dada
        elementos = driver.find_elements(By.CLASS_NAME, clase)
        
        for elemento in elementos:
            if etiqueta in elemento.text:
                texto = elemento.text.strip()
                return texto.split(":")[-1].strip() if ':' in texto else texto
        return 'No encontrado'
    except Exception as e:
        print(f"Error al buscar clase {clase}: {e}")
        return 'No encontrado'

def obtener_contenido_selenium(driver):
    """
    Extrae el contenido principal de la página web usando Selenium.
    
    Args:
    - driver: WebDriver de Selenium.
    
    Returns:
    - str: el contenido de la página o 'No encontrado' si no se encuentra.
    """
    try:
        contenido_div = driver.find_element(By.CLASS_NAME, 'entry-content')
        parrafos = contenido_div.find_elements(By.TAG_NAME, 'p')
        contenido_texto = "\n".join([p.text.strip() for p in parrafos])
        return contenido_texto
    except Exception as e:
        print(f"Error al obtener el contenido: {e}")
        return 'No se encontró el div de contenido'


def obtener_fecha_publicacion_selenium(driver):
    """
    Extrae la fecha de publicación de la página web usando Selenium.
    
    Args:
    - driver: WebDriver de Selenium.
    
    Returns:
    - str: la fecha de publicación o 'No encontrado' si no se encuentra.
    """
    try:
        fecha_div = driver.find_element(By.CLASS_NAME, 'cs-entry__post-meta')
        fecha = fecha_div.find_element(By.CLASS_NAME, 'cs-meta-date').text.strip()
        return fecha
    except Exception as e:
        print(f"Error al obtener la fecha de publicación: {e}")
        return 'No encontrado'


def obtener_header_info_selenium(driver):
    """
    Extrae el título, las categorías y la fecha de publicación desde el div '.cs-entry__header-info' usando Selenium.
    
    Args:
    - driver: WebDriver de Selenium.
    
    Returns:
    - tuple: (header_title, header_categories, header_date)
    """
    try:
        header_info = driver.find_element(By.CLASS_NAME, 'cs-entry__header-info')

        # Extraer título
        header_title = header_info.find_element(By.TAG_NAME, 'h1').text.strip()

        # Extraer categorías
        category_elements = header_info.find_elements(By.CSS_SELECTOR, '.cs-entry__category a')
        header_categories = ", ".join([cat.text.strip() for cat in category_elements])

        # Extraer fecha
        header_date = header_info.find_element(By.CLASS_NAME, 'cs-meta-date').text.strip()

        return header_title, header_categories, header_date
    except Exception as e:
        print(f"Error al obtener la información del header: {e}")
        return 'No encontrado', 'No encontrado', 'No encontrado'


# Crear el driver de Selenium
driver = create_driver(headless=False)

# Llamamos a la función para obtener la información
df_resultado = obtener_informacion_sitios_selenium(driver, cdc[3:6])

# Mostramos el DataFrame con los resultados
df_resultado

Obteniendo datos de: https://www.cdc.cl/publicaciones/historia/discurso-del-presidente-patricio-aylwin-cuando-nacio-cdc/
Error al obtener la fecha de publicación: Message: no such element: Unable to locate element: {"method":"css selector","selector":".cs-entry__post-meta"}
  (Session info: chrome=131.0.6778.205); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7F15CFB05+28789]
	(No symbol) [0x00007FF7F15386E0]
	(No symbol) [0x00007FF7F13D592A]
	(No symbol) [0x00007FF7F142930E]
	(No symbol) [0x00007FF7F14295FC]
	(No symbol) [0x00007FF7F14728A7]
	(No symbol) [0x00007FF7F144F47F]
	(No symbol) [0x00007FF7F146F654]
	(No symbol) [0x00007FF7F144F1E3]
	(No symbol) [0x00007FF7F141A938]
	(No symbol) [0x00007FF7F141BAA1]
	GetHandleVerifier [0x00007FF7F190933D+3410093]
	GetHandleVerifier [0x00007FF7F191E7DD+3497293]
	GetHandleVerifier [0x00007FF7F1912A73+344880

,Enlace,Título,Autor,Colección,Contenido,Fecha de publicación,Título Header,Categorías Header,Fecha Header,Título Ficha,Autor Ficha,Colección Ficha
0,https://www.cdc.cl/publicaciones/historia/disc...,No encontrado,,,"Hace más de una década, cuando se inauguró el ...",No encontrado,No encontrado,No encontrado,No encontrado,Ficha del documento,,
1,https://www.cdc.cl/publicaciones/bl/boletin-le...,No encontrado,Boletín Legislativo N°24.,Boletín Legislativo N°24.,Revisa nuestro Boletín Legislativo de esta sem...,No encontrado,No encontrado,No encontrado,No encontrado,Ficha del documento,Boletín Legislativo N°24.,Boletín Legislativo N°24.
2,https://www.cdc.cl/publicaciones/bl/boletin-le...,No encontrado,Boletín Legislativo N°23.,Boletín Legislativo N°23.,Revisa nuestro Boletín Legislativo de esta sem...,No encontrado,No encontrado,No encontrado,No encontrado,Ficha del documento,Boletín Legislativo N°23.,Boletín Legislativo N°23.


In [55]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import re

# ... (funciones create_driver, obtener_contenido_selenium, obtener_fecha_publicacion_selenium, obtener_header_info_selenium se mantienen igual)

def obtener_valor_ficha(driver, etiqueta_buscada):
    """Extrae el valor de un campo específico dentro de la Ficha del documento usando XPath."""
    try:
        # XPath corregido y robusto: maneja diferentes estructuras y espacios en blanco
        xpath = f"//div[.//h3[contains(.,'Ficha del documento')]]//p[strong[contains(.,'{etiqueta_buscada}')]]/following-sibling::div[1]//p"

        # Espera explícita con mensaje de error más claro
        elemento = WebDriverWait(driver, 0.3).until(
            EC.presence_of_element_located((By.XPATH, xpath)),
            message=f"No se encontró el elemento con XPath: {xpath} para la etiqueta: {etiqueta_buscada}"
        )

        texto = elemento.text.strip()
        # Limpieza adicional para "Palabras clave" (maneja múltiples tags)
        if etiqueta_buscada == "Palabras clave":
            texto = re.sub(r"\s*,\s*", ", ", texto) # Reemplaza múltiples espacios y comas por una coma y un espacio
            texto = texto.replace(" ,", ",") # Elimina la coma inicial si existe
            
        return texto

    except TimeoutException as te:
        print(f"Timeout al buscar '{etiqueta_buscada}': {te.msg}") # Imprime el mensaje del TimeoutException
        return "No encontrado (Timeout)"
    except NoSuchElementException:
        print(f"No se encontró la etiqueta '{etiqueta_buscada}'.")
        return "No encontrado (Elemento no existe)"
    except Exception as e:
        print(f"Error general al obtener '{etiqueta_buscada}': {e}")
        return "No encontrado (Error General)"


def obtener_informacion_ficha_selenium(driver):
    """Extrae la información completa de la Ficha del documento."""
    ficha_info = {}
    etiquetas = ["Título", "Autor(es)", "Colección", "Tema(s)", "Fecha", "Palabras clave"]
    for etiqueta in etiquetas:
        ficha_info[etiqueta] = obtener_valor_ficha(driver, etiqueta)
    return ficha_info

def obtener_informacion_sitios_selenium(driver, df):
    """Función principal para iterar sobre los enlaces y extraer la información."""
    resultados = []
    for index, row in df.iterrows():
        enlace = row['Enlace']
        print(f"Obteniendo datos de: {enlace}")
        try:
            driver.get(enlace)
            time.sleep(1)  # Espera aumentada a 5 segundos

            # ... (resto del código para obtener otros datos: título, autor, etc.)

            ficha_info = obtener_informacion_ficha_selenium(driver)
            resultados.append({
                # ... (resto de los datos)
                **ficha_info  # Incluye los datos de la ficha
            })
        except Exception as e:
            print(f"Error al obtener el sitio {enlace}: {e}")
            resultados.append({
                'Enlace': enlace, # ... (resto de los datos con valor None)
                'Título': None, 'Autor(es)': None, 'Colección': None, 'Tema(s)': None, 'Fecha': None, 'Palabras clave': None
            })

    df_resultado = pd.DataFrame(resultados)
    print(df_resultado.head())
    return df_resultado

# ... (resto del código)

driver = create_driver(headless=False)
df_resultado = obtener_informacion_sitios_selenium(driver, cdc[3:6])
df_resultado

Obteniendo datos de: https://www.cdc.cl/publicaciones/historia/discurso-del-presidente-patricio-aylwin-cuando-nacio-cdc/
Timeout al buscar 'Título': No se encontró el elemento con XPath: //div[.//h3[contains(.,'Ficha del documento')]]//p[strong[contains(.,'Título')]]/following-sibling::div[1]//p para la etiqueta: Título
Timeout al buscar 'Autor(es)': No se encontró el elemento con XPath: //div[.//h3[contains(.,'Ficha del documento')]]//p[strong[contains(.,'Autor(es)')]]/following-sibling::div[1]//p para la etiqueta: Autor(es)
Timeout al buscar 'Colección': No se encontró el elemento con XPath: //div[.//h3[contains(.,'Ficha del documento')]]//p[strong[contains(.,'Colección')]]/following-sibling::div[1]//p para la etiqueta: Colección
Timeout al buscar 'Tema(s)': No se encontró el elemento con XPath: //div[.//h3[contains(.,'Ficha del documento')]]//p[strong[contains(.,'Tema(s)')]]/following-sibling::div[1]//p para la etiqueta: Tema(s)
Timeout al buscar 'Fecha': No se encontró el elemento 

,Título,Autor(es),Colección,Tema(s),Fecha,Palabras clave
0,No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout)
1,No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout)
2,No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout),No encontrado (Timeout)
